In [7]:
import torch
import tensorly as tl
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import QuantLib as ql
import pysabr
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
from scipy.optimize import minimize

%matplotlib inline

In [6]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf

In [8]:
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F

from tensorly.tenalg import inner

In [9]:
tl.set_backend('pytorch')
batch_size = 64
device = 'cpu'
kf = KFold(n_splits=4, shuffle=True)

sabr_params = ["Strike", "Tenor", "Alpha", "Beta", "Nu", "Rho"]

In [35]:
# scaler = MinMaxScaler()

raw_data = pd.read_csv('fdm_hagan_vol_train.csv')
test_raw_data = pd.read_csv('fdm_hagan_vol_test.csv')
raw_data["Error"] = (raw_data["FDM_vol"] - raw_data["Hagan_vol"])
test_raw_data["Error"] = (test_raw_data["FDM_vol"] - test_raw_data["Hagan_vol"])

X = raw_data.drop(["FDM_vol", "Hagan_vol", "Error"], axis=1).values
y = raw_data["Error"].values
X_test = test_raw_data.drop(["FDM_vol", "Hagan_vol", "Error"], axis=1).values
y_test = test_raw_data["Error"].values

# scaler.fit(X)
# X = scaler.transform(X)
# X_test = scaler.transform(X_test)


# for train_index, test_index in kf.split(X):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

In [36]:
X = torch.FloatTensor(X)
y = torch.FloatTensor(y)
y = torch.reshape(y, shape=(-1,1))

X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test)
y_test = torch.reshape(y_test, shape=(-1,1))

In [37]:
train_dataset = torch.utils.data.TensorDataset(X, y)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class HaganDiffNNModel(train_loader, test_loader):
    def __init__(self):
        super(HaganDiffNNModel, self).__init__()
        self.model = nn.Sequential(nn.Linear(6, 20, device=device),
                              nn.Linear(20,100),
                              nn.Linear(100,400),
                              nn.Sigmoid(),
                              nn.Linear(400,10),           
                              nn.Tanh(),
                              nn.Linear(10,1),
                              nn.Tanh()
        )
        self.optimizer = optim.Adam(model.parameters(),lr=0.001)
        self.n_epochs = 2000
        self.batch_size = 64
    
    def hagan(self, X):
        return np.array([ql.sabrVolatility(row[0], 1, *row[1:]) for row in X])
    
    def train(n_epoch):
        self.model.train()
        for i, (features, error) in enumerate(train_loader):
            features, error = features.to(device), error.to(device)
            self.optimizer.zero_grad()
            output = model(features)
            loss = torch.nn.functional.mse_loss(output,error)
            loss.backward()
            self.optimizer.step()
            if i % 1000 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    self.n_epoch, i * len(features), len(train_loader.dataset),
                    100. * i / len(train_loader), loss))
    
    def test():
        self.model.eval()
        test_loss = 0
        correct = 0
        for features, error in test_loader:
            features, error = features.to(device), error.to(device)
            output = self.model(features)
            test_loss = torch.nn.functional.mse_loss(output,error)
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(error.data.view_as(pred)).cpu().sum()

        test_loss /= len(test_loader.dataset)
        print('mean: {}'.format(test_loss))
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
           100. * correct / len(test_loader.dataset)))

    # def forward(self, x):
    #     x = F.relu(F.max_pool2d(self.conv1(x), 2))
    #     x = F.relu(F.max_pool2d(self.conv2(x), 2))
    #     x = self.trl(x)
    #     return F.log_softmax(x)


model = HaganDiffNNModel()
model = model.to(device)

In [31]:
model = nn.Sequential(nn.Linear(6, 20, device=device),
                      nn.Linear(20,100),
                      nn.Linear(100,400),
                      nn.Sigmoid(),
                      nn.Linear(400,10),           
                      nn.Tanh(),
                      nn.Linear(10,1),
                      nn.Tanh()
)

optimizer = optim.Adam(model.parameters(),lr=0.001)
n_epochs = 2000
# batch_size = 64

model = model.to(device)


In [30]:
def train():
    model.train()
    for i, (features, error) in enumerate(train_loader):
        features, error = features.to(device), error.to(device)
        optimizer.zero_grad()
        output = model(features)
        loss = torch.nn.functional.mse_loss(output,error)
        loss.backward()
        optimizer.step()
        if i % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(features), len(train_loader.dataset),
                100. * i / len(train_loader), loss))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for features, error in test_loader:
        features, error = features.to(device), error.to(device)
        output = model(features)
        test_loss = torch.nn.functional.mse_loss(output,error)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(error.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('mean: {}'.format(test_loss))
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
       100. * correct / len(test_loader.dataset)))

for epoch in range(1, 20):
    train(epoch)
    test()


Train Epoch: 1 [0/937295 (0%)]	Loss: 1.286150
Train Epoch: 1 [64000/937295 (7%)]	Loss: 0.477125
Train Epoch: 1 [128000/937295 (14%)]	Loss: 0.078490
Train Epoch: 1 [192000/937295 (20%)]	Loss: 0.265546
Train Epoch: 1 [256000/937295 (27%)]	Loss: 0.350423
Train Epoch: 1 [320000/937295 (34%)]	Loss: 0.176431
Train Epoch: 1 [384000/937295 (41%)]	Loss: 0.327776
Train Epoch: 1 [448000/937295 (48%)]	Loss: 0.113970
Train Epoch: 1 [512000/937295 (55%)]	Loss: 0.246343
Train Epoch: 1 [576000/937295 (61%)]	Loss: 0.186801
Train Epoch: 1 [640000/937295 (68%)]	Loss: 0.219283
Train Epoch: 1 [704000/937295 (75%)]	Loss: 0.316831
Train Epoch: 1 [768000/937295 (82%)]	Loss: 0.386490
Train Epoch: 1 [832000/937295 (89%)]	Loss: 0.459918
Train Epoch: 1 [896000/937295 (96%)]	Loss: 0.477739
mean: 1.0904304872383364e-05

Test set: Average loss: 0.0000, Accuracy: 0/50000 (0%)

Train Epoch: 2 [0/937295 (0%)]	Loss: 0.168692
Train Epoch: 2 [64000/937295 (7%)]	Loss: 0.337270
Train Epoch: 2 [128000/937295 (14%)]	Loss: 0.7

In [39]:
def train():
    model.train()
    for i, (features, error) in enumerate(train_loader):
        features, error = features.to(device), error.to(device)
        optimizer.zero_grad()
        output = model(features)
        loss = torch.nn.functional.mse_loss(output,error)
        loss.backward()
        optimizer.step()
        if i % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(features), len(train_loader.dataset),
                100. * i / len(train_loader), loss))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for features, error in test_loader:
        features, error = features.to(device), error.to(device)
        output = model(features)
        test_loss = torch.nn.functional.mse_loss(output,error)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(error.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('mean: {}'.format(test_loss))
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
       100. * correct / len(test_loader.dataset)))

for epoch in range(1, 20):
    train()
    test()

Train Epoch: 1 [0/937295 (0%)]	Loss: 0.760014
Train Epoch: 1 [64000/937295 (7%)]	Loss: 0.009469
Train Epoch: 1 [128000/937295 (14%)]	Loss: 0.004458
Train Epoch: 1 [192000/937295 (20%)]	Loss: 0.005546
Train Epoch: 1 [256000/937295 (27%)]	Loss: 0.001988
Train Epoch: 1 [320000/937295 (34%)]	Loss: 0.003279
Train Epoch: 1 [384000/937295 (41%)]	Loss: 0.001299
Train Epoch: 1 [448000/937295 (48%)]	Loss: 0.011267
Train Epoch: 1 [512000/937295 (55%)]	Loss: 0.008680
Train Epoch: 1 [576000/937295 (61%)]	Loss: 0.002495
Train Epoch: 1 [640000/937295 (68%)]	Loss: 0.000654
Train Epoch: 1 [704000/937295 (75%)]	Loss: 0.000714
Train Epoch: 1 [768000/937295 (82%)]	Loss: 0.001763
Train Epoch: 1 [832000/937295 (89%)]	Loss: 0.002349
Train Epoch: 1 [896000/937295 (96%)]	Loss: 0.001596
mean: 1.4340043819061066e-08

Test set: Average loss: 0.0000, Accuracy: 0/50000 (0%)

Train Epoch: 2 [0/937295 (0%)]	Loss: 0.009022
Train Epoch: 2 [64000/937295 (7%)]	Loss: 0.003046
Train Epoch: 2 [128000/937295 (14%)]	Loss: 0.0